In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Udem/DKL

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Udem/DKL


In [3]:
!pwd

/content/drive/MyDrive/Udem/DKL


In [3]:
!pip install gpytorch
!pip install optuna

In [4]:
import pandas as pd
import numpy as np
import os
import torch
import torch.optim as optim
from features import feature_prep_air_quality, inverse_transform, format_predictions, calculate_metrics
# from features import SequenceEncoder
from model import LSTMModel, ExactGPLayer,DeepGP, MLPModel, Bias, GPModel, WaveNet, ConvRNN
from run_exp import Optimization

import matplotlib.pyplot as plt
import torch.nn as nn
import gpytorch
from gpytorch.mlls import VariationalELBO, AddedLossTerm
from gpytorch.mlls import DeepApproximateMLL
import time
import optuna
import pickle
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestRegressor
# from mapie.metrics import (regression_coverage_score, regression_mean_width_score)
# from mapie.regression import MapieRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from scipy.stats import randint
from sklearn.metrics import mean_absolute_error
import random
from sklearn.model_selection import train_test_split

In [5]:
torch.__version__

'2.0.1+cu118'

In [6]:
!python -V

Python 3.10.12


In [7]:
!nvidia-smi

Sat Jul  8 01:24:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
# df = pd.read_csv('data.csv')
df = pd.read_excel('AirQualityUCI.xlsx')
df['hour'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.hour
df['month'] = pd.DatetimeIndex(df['Date']).month

In [15]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
def get_model(model, model_params):
    models = {
        "convrnn": ConvRNN,
        "wavenet": WaveNet,
        "lstm": LSTMModel,
        "mlp": MLPModel,
        "bias": Bias
    }
    return models.get(model.lower())(**model_params)

def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_uniform_(m.weight.data).to(device)
        nn.init.constant_(m.bias.data, 0).to(device)

def get_model_likelihood_mll(train_x=None, train_y=None, batch_size=64, input_dim=32, device=torch.device("cpu"), mode = 'ExactGP', kernel='RBF'):
    # noise_prior = gpytorch.priors.Prior()
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    if (train_x is None): train_x = torch.ones(batch_size, input_dim).to(device)
    if (train_y is None): train_y = torch.ones(batch_size).to(device)
    model = ExactGPLayer(train_x,train_y,likelihood, kernel=kernel)
    return model, likelihood



In [16]:
#!unset LD_LIBRARY_PATH
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)


In [19]:
X_train.shape

(5393, 14)

In [18]:
start_time = time.time()

batch_size = 256
X_train, y_train, train_features, train_targets, X_test,y_test, scaler, scaler1, train_loader,val_loader, test_loader, train_loader_one, test_loader_one = feature_prep_air_quality(df, batch_size=batch_size)

device = torch.device("cpu")
input_dim = X_train.shape[1]
output_dim = 1
hidden_dim = 100
layer_dim = 3
kernel = 1
out_channels = 24

dropout = 0.2
n_epochs = 200
learning_rate = 0.001
weight_decay = 1e-6

model_params = {'input_dim': input_dim,
                'hidden_dim': hidden_dim,
                'layer_dim': layer_dim,
                'output_dim': output_dim,
                'dropout_prob': dropout,
                'device': device}

mlp_params = {'input_dim': input_dim, 'hidden_dim': hidden_dim, 'output_dim': output_dim}
bias_params = {'input_dim': output_dim, 'output_dim': output_dim, 'device': device}
convrnn_params = {'input_dim':input_dim, 'timesteps':X_train.shape[1], 'output_dim':output_dim, 'kernel_size1':7, 'kernel_size2':5, 'kernel_size3':3,
              'n_channels1':32, 'n_channels2':32, 'n_channels3':32, 'n_units1':32, 'n_units2':32, 'n_units3':32}
mode = 'LSTM_ExactGP'
#options:{LSTM_ExactGP, DeepGP, ExactGP, mlp_ExactGP}

if mode == 'LSTM_ExactGP': # LSTM_ExactGP
    feature_extractor = get_model('lstm', model_params)
    feature_extractor.to(device)
    model, likelihood = get_model_likelihood_mll(train_x=None, train_y=None, batch_size=batch_size,
                                                  input_dim=output_dim, device=device, mode=mode)
    loss_fn = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    feature_extractor.apply(initialize_weights)
    model.to(device)
else:

    feature_extractor = get_model('mlp', mlp_params)
    feature_extractor.to(device)

    model, likelihood = get_model_likelihood_mll(train_x=None, train_y=None, batch_size=batch_size,input_dim=output_dim, device=device, mode=mode)
    model.to(device)
    likelihood.to(device)
    loss_fn = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    feature_extractor.apply(initialize_weights)


likelihood.to(device)
lr = 0.00177040768495762

optimizer = torch.optim.Adam([
    {'params': feature_extractor.parameters(), 'lr': lr},
    {'params': model.covar_module.parameters(), 'lr': lr * 0.1},
    {'params': model.mean_module.parameters(), 'lr': lr},
    {'params': likelihood.parameters(), 'lr': lr}, ])

opt = Optimization(feature_extractor=feature_extractor, model=model, likelihood=likelihood, loss_fn=loss_fn, optimizer=optimizer, device=device, mode=mode)
opt.train(train_loader,val_loader, batch_size=batch_size, n_epochs=n_epochs, n_features=input_dim)
opt.plot_losses()

predictions, values,variance, lower_pred, upper_pred, mean_f, lower_f, upper_f= opt.evaluate(train_loader,test_loader_one, batch_size=1, n_features=input_dim)

df_result = format_predictions(predictions,values,variance, lower_pred, upper_pred, mean_f, lower_f, upper_f, scaler1)
df_result.to_csv('results.csv', index=True)

plt.figure(figsize=(20, 4))
plt.plot(df_result.tail(200)['prediction'], label="predicted")
plt.plot(
    df_result.tail(200)['actual'], label="actual"
)

plt.xlabel("Date")
plt.ylabel("Target values")
plt.legend()
plt.savefig("Predictions.png")
plt.close()
plt.figure(figsize=(20, 4))
plt.plot(
    df_result.head(200)['prediction'], label="predicted"
)

plt.plot(
    df_result.head(200)['actual'], label="actual"
)

plt.xlabel("Date")
plt.ylabel("Target values")
plt.legend()
plt.savefig("Predictions1.png")
plt.close()

print('Overall time taken(in hours):',(time.time()-start_time)/3600)
result_metrics = calculate_metrics(df_result)
print('metrics',result_metrics)

plt.figure(figsize=(20, 6))
plt.plot(df_result.index, df_result['prediction'], 'b')
plt.fill_between(df_result.index, df_result['lower_pred'], df_result['upper_pred'], alpha=0.5)
plt.legend(['Mean', '95% Confidence'], fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=16)
plt.tick_params(axis='both', which='minor', labelsize=16)
plt.tight_layout()
plt.savefig("Predictions2.png")
plt.close()


Identified outliers: 368
Non-outlier observations: 8989


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 256])) that is different to the input size (torch.Size([256])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[1/200] Training loss:  0.9649 - noise =  6.748170e-01 	 train_mse:  1.0075
[2/200] Training loss:  0.8057 - noise =  6.544352e-01 	 train_mse:  1.0046
[3/200] Training loss:  0.7657 - noise =  6.341863e-01 	 train_mse:  1.0027
[4/200] Training loss:  0.7426 - noise =  6.147937e-01 	 train_mse:  1.0016
[5/200] Training loss:  0.7245 - noise =  5.960623e-01 	 train_mse:  1.0017
[6/200] Training loss:  0.7084 - noise =  5.780129e-01 	 train_mse:  1.0021
[7/200] Training loss:  0.6983 - noise =  5.606295e-01 	 train_mse:  1.0026
[8/200] Training loss:  0.6800 - noise =  5.436474e-01 	 train_mse:  1.0031
[9/200] Training loss:  0.6616 - noise =  5.270418e-01 	 train_mse:  1.0039
[10/200] Training loss:  0.6482 - noise =  5.108765e-01 	 train_mse:  1.0042
[20/200] Training loss:  0.4764 - noise =  3.684361e-01 	 train_mse:  1.0119
[30/200] Training loss:  0.3103 - noise =  2.605737e-01 	 train_mse:  1.0219
[40/200] Training loss:  0.1474 - noise =  1.824932e-01 	 train_mse:  1.0243
[50/200]

In [ ]:
metrics {'mae': 3.9410331, 'mse': 4.550951830896332, 'r2': 0.4259817749709902} - epoch 50 spectral

In [ ]:
def objective(trial):

    params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'batch_size': trial.suggest_categorical("batch", [64, 128, 256]),
              'dropout':trial.suggest_loguniform('dropout', 0.1, 0.4),
              'kernel':trial.suggest_categorical("kernel", ['RBF', 'spectral', 'cosine', 'matern', 'BNCosSim']),
              }
    features = ['draft_aft_telegram', 'draft_fore_telegram', 'stw',
    'diff_speed_overground', 'awind_vcomp_provider', 'awind_ucomp_provider',
    'rcurrent_vcomp', 'rcurrent_ucomp', 'comb_wind_swell_wave_height',
    'timeSinceDryDock']
    X_train = train[features]
    y_train = train['power']
    X_test = test[features]
    y_test = test['power']

    X_train, y_train, train_features, train_targets, X_test,y_test, scaler, scaler1, train_loader,val_loader, test_loader, train_loader_one, test_loader_one = feature_prep(train, val, test, batch_size=params["batch_size"])
    device = torch.device("cuda")
    input_dim = X_train.shape[1]
    output_dim = 1
    hidden_dim = 100
    layer_dim = 3
    kernel = 1
    out_channels = 24
    mode = 'LSTM_ExactGP'
    dropout = params['dropout']
    n_epochs = 1
    learning_rate = 0.001
    weight_decay = 1e-6

    model_params = {'input_dim': input_dim,
                    'hidden_dim': hidden_dim,
                    'layer_dim': layer_dim,
                    'output_dim': output_dim,
                    'dropout_prob': dropout,
                    'device': device}
    feature_extractor = get_model('lstm', model_params)
    feature_extractor.to(device)
    model, likelihood = get_model_likelihood_mll(train_x=None, train_y=None, batch_size=params["batch_size"],
                                                  input_dim=output_dim, device=device, mode=mode, kernel = params["kernel"])
    loss_fn = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    feature_extractor.apply(initialize_weights)
    model.to(device)
    # optim_params = list(feature_extractor.parameters()) + list(model.covar_module.parameters()) + list(model.mean_module.parameters()) + list(likelihood.parameters())
    # learn_params = list(lr) + list(lr*0.1) + list(lr) + list(lr)
    # optimizer = getattr(torch.optim, params['optimizer'])([feature_extractor.parameters(), lr= params['learning_rate']])

    optimizer = getattr(torch.optim, params['optimizer'])([
    {'params': feature_extractor.parameters(), 'lr': params['learning_rate']},
    {'params': model.covar_module.parameters(), 'lr': params['learning_rate'] * 0.1},
    {'params': model.mean_module.parameters(), 'lr': params['learning_rate']},
    {'params': likelihood.parameters(), 'lr': params['learning_rate']}, ])

    opt = Optimization(feature_extractor=feature_extractor, model=model, likelihood=likelihood, loss_fn=loss_fn, optimizer=optimizer, device=device, mode=mode)
    opt.train(train_loader,val_loader, batch_size=params["batch_size"], n_epochs=n_epochs, n_features=input_dim)
    predictions, values,variance, lower_pred, upper_pred, mean_f, lower_f, upper_f = opt.evaluate(train_loader, test_loader_one, batch_size=1, n_features=input_dim)
    df_result = format_predictions(predictions,values,variance, lower_pred, upper_pred, mean_f, lower_f, upper_f, scaler1)
    # print('values',np.concatenate(values, axis=0).ravel().shape)
    # print('prediction',np.concatenate(predictions, axis=0).ravel().shape)
    # mae_mean = mean_absolute_error(np.concatenate(values, axis=0).ravel(), np.concatenate(predictions, axis=0).ravel())
    result_metrics = calculate_metrics(df_result)
    return result_metrics["mae"]


In [ ]:
%%time
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
# 1epoch: 'mae': 1260.5171, 'mse': 1578.8270171237887, 'r2': 0.9137536772367476 batch active learning
#100 epochs: 'mae': 5453.195, 'mse': 6785.376923944608, 'r2': 0.2265047417595234
#200 epochs metrics {'mae': 4877.8594, 'mse': 6272.694158015358, 'r2': 0.3389748865544878}
#100 epochs with transformer encoder metrics {'mae': 6439.92, 'mse': 7715.523313424696, 'r2': -9.323326833987622e-05}
#Area under Mean Square error (MSE) and F1 retention curves [Malinin, 2019, Malinin et al., 2021] is used to assess jointly the robustness to distributional shift and uncertainty quality.